This notebook aims to recreate an annealer machine running simulated annealing.

In [2]:
import numpy as np
import numba as nb
import time

In [3]:
def default_temp_schedule(num_iter, temp_start, decay_rate, mode='EXPONENTIAL'):
    """
    Generates a list of temperatures for annealing algorithms.
    
    Parameters:
        num_iter (int): Length of the list.
        temp_start (number): Value of the first element in the returned list.
        decay_rate (number): Multiplier for changing the temperature during annealing.
        mode (string, default='EXPONENTIAL'):
            Three modes are possible. Note the accepted ranges for decay_rate are different.
            'EXPONENTIAL':  T[i+1] = T[i] * (1 - decay_rate)           # 0 <= decay_rate < 1
            'INVERSE':      T[i+1] = T[i] * (1 - decay_rate * T[i])    # 0 <= decay_rate < 1/temp_start
            'INVERSE_ROOT': T[i+1] = T[i] * (1 - decay_rate * T[i]**2) # 0 <= decay_rate < 1/temp_start**2
    
    Return: temp_schedule (list[number])
    """
    
    if mode == 'EXPONENTIAL':
        if 0 <= decay_rate < 1:
            TS = [temp_start]
            for _ in range(num_iter - 1):
                TS.append(TS[-1] * (1 - decay_rate))
            return TS
        else:
            raise ValueError("decay_rate out of accepted range")
    elif mode == 'INVERSE':
        if 0 <= decay_rate < 1/temp_start:
            TS = [temp_start]
            for _ in range(num_iter - 1):
                TS.append(TS[-1] * (1 - decay_rate * TS[-1]))
            return TS
        else:
            raise ValueError("decay_rate out of accepted range")
    elif mode == 'INVERSE_ROOT':
        if 0 <= decay_rate < 1/temp_start**2:
            TS = [temp_start]
            for _ in range(num_iter - 1):
                TS.append(TS[-1] * (1 - decay_rate * TS[-1]**2))
            return TS
        else:
            raise ValueError("decay_rate out of accepted range")
    else:
        raise ValueError("mode not supported")

In [4]:
@nb.njit(parallel=False)
def one_MA_run(Q_matrix, temp_schedule, scaling_schedule, dropout_schedule, ansatz_state=None):
    """
    One momentum annealing run over the full temperature schedule.
    
    Parameters:
        Q_matrix (2-D array of float64): The matrix representing the local and coupling field of the problem.
        temp_schedule (list[float64]): The annealing temperature schedule.
                                       The number of iterations is implicitly the length of temp_schedule.
        scaling_schedule (list[float64]): The momentum factor scaling schedule.
        dropout_schedule (list[float64]): The momentum factor dropout rate schedule.
        ansatz_state (1-D array of bool, default=None): The boolean vector representing the initial state.
                                                        If None, a random state is chosen.
    
    Return: final_state (1-D array of bool)
    """
    
    if len(temp_schedule) != len(scaling_schedule) or len(temp_schedule) != len(dropout_schedule):
        raise InputError("The three input schedules should have equal lengths.")
    
    # Q_coef[i][j]: local field of i if i==j; coupling strength if i!=j
    Q_coef = Q_matrix + Q_matrix.T - np.diag(Q_matrix)
    N = Q_matrix.shape[0]
    
    big_eigval = max(np.linalg.eigvals(-Q_matrix))
    
    if ansatz_state is None:
        state = (np.random.binomial(1, 0.5, N) == 1)
    else:
        state = ansatz_state
    old_state = state.copy()
    
    for k in range(len(temp_schedule)):
        flip = np.zeros(N, dtype=np.bool_)
        for spin in nb.prange(N):
            delta_E = (1 - 2*old_state[spin]) * (np.sum(Q_coef[spin][state]) + Q_coef[spin, spin] * (1 - state[spin]) + \
                                    scaling_schedule[k] * w[spin] * state[spin] * np.random.binomial(1, dropout_schedule[k]))
            if np.random.binomial(1, np.minimum(np.exp(-delta_E/temp_schedule[k]), 1.)):
                flip[spin] = True
        old_state = state.copy()
        state ^= flip
    
    return state

In [5]:
Q = np.array([[-1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]])
ansatz = np.zeros(4, dtype=np.bool_)
TS = default_temp_schedule(10000, 300., 0.001)

In [42]:
# With numba, not parallelized, first pass
np.random.seed(0)
start_time = time.time()
ans = one_MA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

C:\ProgramData\Anaconda3\lib\site-packages\numba\core\ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'temp_schedule' of function 'one_SA_run'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-40-2ef8f95cc4a7>", line 2:
@nb.njit(parallel=False)
def one_SA_run(Q_matrix, temp_schedule, ansatz_state=None):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


ground state: [ True False False False]; time: 0.6684708595275879 s


In [43]:
# With numba, not parallelized, second pass
np.random.seed(0)
start_time = time.time()
ans = one_MA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [ True False False False]; time: 0.026996374130249023 s


In [46]:
# With numba, parallelized, first pass
np.random.seed(0)
start_time = time.time()
ans = one_MA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

C:\ProgramData\Anaconda3\lib\site-packages\numba\core\ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'temp_schedule' of function 'one_SA_run'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-44-52216e0bac78>", line 2:
@nb.njit(parallel=True)
def one_SA_run(Q_matrix, temp_schedule, ansatz_state=None):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


ground state: [ True False False False]; time: 2.386690855026245 s


In [47]:
# With numba, parallelized, second pass
np.random.seed(0)
start_time = time.time()
ans = one_MA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [ True False False False]; time: 0.09206318855285645 s


In [21]:
Q[0][0]

-1.0

In [22]:
Q[0, 0]

-1.0